In [2]:
import random
from bisect import bisect_left

SEED = 1
LASTTIME = 100;
LAMDAS = [0.01, 0.1, 0.3, 0.5] #the fault generation rates at each layer
I = [1, 2, 3, 4, 5] #layer i 
N = [10, 100, 100, 100, 100] #number of elements for layer i
M = [3, 30, 30, 30, 30] #select number of elements to monitor out of N for layer i
random.seed(SEED)

#Generate N elements for each layer and select m out of 
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

elements = []
elementID = 0
for i in I:
    layer = i - 1
    j = 0
    elements_layer = []
    while j < N[layer]: #generate Ni elements
        elementID = elementID + 1
        elements_layer.append(Element(elementID, i, False))
        j = j + 1
    count = 0
    assert(M[layer] <= N[layer]), "Total elements Ni is too small to reach Mi!"
    while count < M[layer]: #select Mi elements out of Ni 
        randomElement = random.randrange(N[layer])
        if elements_layer[randomElement].monitored == False:
            elements_layer[randomElement].monitored = True
            count = count + 1
    elements.append(elements_layer)
    
    #print(M[layer])
    #print(N[layer])
    #for e in element_layer:
    #    e.infoOut()  
#print(len(elements))

class Fault: #define a fault
 def __init__(self, fID, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is sorted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )
    
def insert(seq, keys, item, keyfunc=lambda v: v):
    """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.

    Based on insert() method in SortedCollection recipe:
    http://code.activestate.com/recipes/577197-sortedcollection/
    """
    k = keyfunc(item)  # Get key.
    i = bisect_left(keys, k)  # Determine where to insert item.
    keys.insert(i, k)  # Insert key of item to keys list.
    seq.insert(i, item)  # Insert the item itself in the corresponding place.
    

evtID = 0
queue = [] #a queue of faults 

for i in I: # for each layer, generate a seqence of faults, the evtTime of each sequence starts from 0
    evtTime = 0
    evtID = evtID + 1
    elements_layer = elements[i-1] #find corresponding elements_layer and element randomly selected to be faulty
    
    elementIndex = random.randrange(len(elements_layer))
    newFault = Fault(evtID, i, evtTime, evtTime, 
                     elements_layer[elementIndex], random.random())
    keys = [fault.evtTime for fault in queue]
    insert(queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert in the order of evtTime
    #newFault.infoOut()

    evtTime = evtTime + random.expovariate(LAMDAS[1])
    while evtTime <= LASTTIME:
        evtID = evtID + 1
        elementIndex = random.randrange(len(elements_layer))
        newFault = Fault(evtID, i, evtTime, evtTime, 
                     elements_layer[elementIndex], random.random())     
        keys = [fault.evtTime for fault in queue]
        insert(queue, keys, newFault, keyfunc=lambda x: x.evtTime)
        #newFault.infoOut()
        evtTime = evtTime + random.expovariate(LAMDAS[1])

#sort the faults in queue according to the evtTime    
#print(len(queue)) 
for f in queue:
    f.infoOut()

countMonitored = 0
for f in queue:
    if f.assocElement.monitored == True:
        countMonitored = countMonitored + 1
print("The ratio of monitored elements = " + str(countMonitored/len(queue)))

Fault info: id = 46, layer = 5, initTime = 0, evtTime = 0, assocElementID = 398, monitored = False, impactVector = 0.5343300438262426
Fault info: id = 34, layer = 4, initTime = 0, evtTime = 0, assocElementID = 283, monitored = False, impactVector = 0.460852490153291
Fault info: id = 28, layer = 3, initTime = 0, evtTime = 0, assocElementID = 205, monitored = False, impactVector = 0.29860594962301334
Fault info: id = 13, layer = 2, initTime = 0, evtTime = 0, assocElementID = 55, monitored = False, impactVector = 0.29028432123001946
Fault info: id = 1, layer = 1, initTime = 0, evtTime = 0, assocElementID = 9, monitored = False, impactVector = 0.7773971822959025
Fault info: id = 14, layer = 2, initTime = 1.832863909135774, evtTime = 1.832863909135774, assocElementID = 43, monitored = True, impactVector = 0.5273803990480128
Fault info: id = 2, layer = 1, initTime = 2.300237377347993, evtTime = 2.300237377347993, assocElementID = 1, monitored = False, impactVector = 0.4811018174142402
Fault 